# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [1]:
from copy import deepcopy

class Matrix:
    def __init__(self, n, m=None):
        if isinstance(n, list):
            if not all(isinstance(row, list) for row in n):
                raise ValueError("matrix has to be a list of lists")
            row_lengths = [len(row) for row in n]
            if len(set(row_lengths)) != 1:
                raise ValueError("full rows must have the same # of col")
            self.data = deepcopy(n)
        else:
            self.data = [[0.0 for _ in range(m)] for _ in range(n)]

    def __getitem__(self, idx):
        if isinstance(idx, tuple):
            i, j = idx
            return self.data[i][j]
        return self.data[idx]

    def __setitem__(self, idx, value):
        if isinstance(idx, tuple):
            i, j = idx
            self.data[i][j] = value
        else:
            self.data[idx] = value

    def shape(self):
        return len(self.data), len(self.data[0])

    def transpose(self):
        rows, cols = self.shape()
        return Matrix([[self.data[i][j] for i in range(rows)] for j in range(cols)])

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[self.data[i][n]] for i in range(self.shape()[0])])

    def to_list(self):
        return deepcopy(self.data)

    def block(self, n0, n1, m0, m1):
        return Matrix([row[m0:m1] for row in self.data[n0:n1]])

    def scalarmul(self, c):
        return Matrix([[x * c for x in row] for row in self.data])

    def add(self, other):
        if self.shape() != other.shape():
            raise ValueError("dimension doesnt agree")
        return Matrix([[self.data[i][j] + other.data[i][j] for j in range(self.shape()[1])] for i in range(self.shape()[0])])

    def sub(self, other):
        if self.shape() != other.shape():
            raise ValueError("dimension need to agree in thi one")
        return Matrix([[self.data[i][j] - other.data[i][j] for j in range(self.shape()[1])] for i in range(self.shape()[0])])

    def mat_mult(self, other):
        if self.shape()[1] != other.shape()[0]:
            raise ValueError("shape does not align for multiplying")
        result = Matrix(self.shape()[0], other.shape()[1])
        for i in range(self.shape()[0]):
            for j in range(other.shape()[1]):
                result[i, j] = sum(self.data[i][k] * other.data[k][j] for k in range(self.shape()[1]))
        return result

    def element_mult(self, other):
        if self.shape() != other.shape():
            raise ValueError("the dimensions need to agree on this")
        return Matrix([[self.data[i][j] * other.data[i][j] for j in range(self.shape()[1])] for i in range(self.shape()[0])])

    def equals(self, other):
        return self.data == other.data

    def __eq__(self, other):
        return self.equals(other)

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        return self.mat_mult(other)

    def __rmul__(self, other):
        return self * other

def constant(n, m, c):
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    return Matrix([[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)])


# THE TESTING OF THE CODE 
a = Matrix([[1, 2], [3, 4]])
b = Matrix([[2, 0], [1, 2]])
c = Matrix([[0, 1], [4, 3]])
d = eye(2)

print("(ab)c == a(bc):", (a * b) * c == a * (b * c))
print("a(b+c) == ab + ac:", a * (b + c) == (a * b + a * c))
print("ab != ba:", a * b != b * a)
print("ad == a:", a * d == a)


(ab)c == a(bc): True
a(b+c) == ab + ac: True
ab != ba: True
ad == a: True
